In [48]:
## import
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [29]:
## load data
raw_data = pd.read_csv("spam.csv", encoding="Latin")

In [30]:
## visualize (print) data
raw_data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [31]:
## variable for v1 and v2
tag = raw_data["v1"]
message = raw_data["v2"]

In [32]:
## cleaning data (message):
# 1. all lower case 
# 2. remove numbers and special characters
# 3. remove stop words 
# 4. stemming words

## stemming words
stemmer = SnowballStemmer("english")

## remove stop words
stop_word = stopwords.words("english")

In [41]:
## function that cleans the message
def message_cleaner(input_message):
    output_message = []
    for each_message in input_message:
        each_temp_message = each_message.lower() # lower case
        each_temp_message = re.findall('[a-zA-Z]+', each_message) # regularizing message
        each_temp_message = [word for word in each_temp_message if not word in stop_word] # removes stop words
        each_temp_message = " ".join(each_temp_message) # join words together
        clean_message = stemmer.stem(each_temp_message) # stemming message
        output_message.append(clean_message)
    return output_message

In [42]:
clean_text = message_cleaner(message)

## test
print(message[0])
clean_text[0]

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [43]:
## vectorize data
cv = CV(ngram_range = (0,2), encoding="latin", max_features=20000) # counts the frequency of individual words and phrases
X = cv.fit_transform(clean_text)
features = cv.get_feature_names()
dtm = pd.DataFrame(X.toarray(), columns = features)
dtm = dtm.drop([""], axis = 1)

In [44]:
## test
dtm.head()

,aah,aaniye pudunga,aaooooright,aathi,aathi dear,aathi love,abi,abi how,abi hw,ability,...,zac,zaher,zed,zed per,zed profit,zindgi,zindgi wo,zoe,zs,zs subscription
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
## train and test data

X_train, X_test, y_train, y_test = train_test_split(dtm, tag, test_size = 0.1, random_state = 40)
print("Running Model!")
clfrf = RandomForestClassifier(200, n_jobs = -1, bootstrap = True)
clfrf.fit(X_train, y_train)
prediction = clfrf.predict(X_test)
print(accuracy_score(prediction,y_test))

Running Model!
0.974910394265233
